In [55]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error

In [56]:
# Load the sales data
sales_data = pd.read_csv(r'F:\GUVI\Project\Dominos - Predictive Purchase Order System\Cleaned_DATA\Pizza_sales.csv', index_col=0)
# Load the ingredients data
ingredients_data = pd.read_csv(r'F:\GUVI\Project\Dominos - Predictive Purchase Order System\Cleaned_DATA\ingredients_data.csv', index_col=0)

In [135]:
ingredients_data

,pizza_name_id,pizza_name,pizza_ingredients,Items_Qty_In_Grams
0,bbq_ckn_l,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,bbq_ckn_l,The Barbecue Chicken Pizza,Red Peppers,15.0
2,bbq_ckn_l,The Barbecue Chicken Pizza,Green Peppers,20.0
3,bbq_ckn_l,The Barbecue Chicken Pizza,Tomatoes,30.0
4,bbq_ckn_l,The Barbecue Chicken Pizza,Red Onions,60.0
...,...,...,...,...
513,veggie_veg_s,The Vegetables + Vegetables Pizza,Green Peppers,20.0
514,veggie_veg_s,The Vegetables + Vegetables Pizza,Red Onions,20.0
515,veggie_veg_s,The Vegetables + Vegetables Pizza,Zucchini,10.0
516,veggie_veg_s,The Vegetables + Vegetables Pizza,Spinach,15.0


In [57]:
sales_data.isnull().sum()

pizza_id             0
order_id             0
pizza_name_id        0
quantity             0
order_date           0
order_time           0
unit_price           0
total_price          0
pizza_size           0
pizza_category       0
pizza_ingredients    0
pizza_name           0
dtype: int64

In [58]:
# Convert order_date to datetime
sales_data['order_date'] = pd.to_datetime(sales_data['order_date'], format='mixed')

In [59]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   pizza_id           48620 non-null  int64         
 1   order_id           48620 non-null  int64         
 2   pizza_name_id      48620 non-null  object        
 3   quantity           48620 non-null  int64         
 4   order_date         48620 non-null  datetime64[ns]
 5   order_time         48620 non-null  object        
 6   unit_price         48620 non-null  float64       
 7   total_price        48620 non-null  float64       
 8   pizza_size         48620 non-null  object        
 9   pizza_category     48620 non-null  object        
 10  pizza_ingredients  48620 non-null  object        
 11  pizza_name         48620 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 4.8+ MB


In [60]:
sales_data.head(5)

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,2015-01-01,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,2015-01-01,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [134]:
sales_data['pizza_name_id'].value_counts()

pizza_name_id
big_meat_s        1811
thai_ckn_l        1365
five_cheese_l     1359
four_cheese_l     1273
classic_dlx_m     1159
                  ... 
mexicana_s         160
calabrese_s         99
ckn_alfredo_s       96
green_garden_l      94
the_greek_xxl       28
Name: count, Length: 91, dtype: int64

In [61]:
sales_data['month'] = sales_data['order_date'].dt.month

In [132]:
sales_data.shape

(48620, 13)

In [62]:
sales_data.head(5)

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name,month
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza,1
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza,1
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza,1
3,4,2,ital_supr_l,1,2015-01-01,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza,1
4,5,2,mexicana_m,1,2015-01-01,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza,1


In [63]:
# Aggregate sales across all pizza types for each date
aggregated_sales = sales_data.groupby('order_date')['quantity'].sum().reset_index()


In [64]:
aggregated_sales.set_index("order_date", inplace=True)
aggregated_sales

,quantity
order_date,
2015-01-01,162
2015-01-02,191
2015-01-03,99
2015-01-04,135
2015-01-05,154
...,...
2015-12-27,89
2015-12-28,102
2015-12-29,80


In [65]:
# Step 2: Split the data
train = aggregated_sales['2015-01-01':'2015-10-31']
test = aggregated_sales['2015-11-01':'2015-12-31']

In [68]:
train

,quantity
order_date,
2015-01-01,162
2015-01-02,191
2015-01-03,99
2015-01-04,135
2015-01-05,154
...,...
2015-10-27,120
2015-10-28,104
2015-10-29,131


In [69]:
test

,quantity
order_date,
2015-11-01,116
2015-11-02,155
2015-11-03,134
2015-11-04,153
2015-11-05,144
2015-11-06,158
2015-11-07,125
2015-11-08,137
2015-11-09,167


In [86]:
def models_result(model_fit):

    # Step 4: Forecast the test set
    test_forecast = model_fit.forecast(steps=len(test))
    mape = mean_absolute_percentage_error(test['quantity'], test_forecast)
    print(f'MAPE: {mape:.5f}')

    # Step 5: Predict next week sales
    next_week_forecast = model_fit.forecast(steps=7)
    next_week_dates = pd.date_range(start='2016-01-01', periods=7)
    
    # Create a DataFrame for the predictions
    predictions = pd.DataFrame({'date': next_week_dates, 'predicted_sales': next_week_forecast})
    predictions.set_index('date', inplace=True)
    print(predictions)

## ARIMA

In [93]:
# Step 3: Fit the ARIMA model
model = ARIMA(train['quantity'], order=(1, 0, 1))  # Parameters can be tuned
ARIMA_model_fit = model.fit()

c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [94]:
models_result(ARIMA_model_fit)

MAPE: 0.16232
            predicted_sales
date                       
2016-01-01       138.269207
2016-01-02       138.287148
2016-01-03       138.304168
2016-01-04       138.320313
2016-01-05       138.335628
2016-01-06       138.350155
2016-01-07       138.363937


c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the n

## SARIMA

In [107]:
model = SARIMAX(train['quantity'], order=(1, 0, 1), seasonal_order=(1, 1, 1, 12))
SARIMAX_model_fit = model.fit()

c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [108]:
models_result(SARIMAX_model_fit)

MAPE: 0.16953
            predicted_sales
date                       
2016-01-01       140.237563
2016-01-02       139.971663
2016-01-03       140.370654
2016-01-04       136.308228
2016-01-05       137.388591
2016-01-06       137.619929
2016-01-07       138.481151


c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\syeda\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the n

# Differenct Approach

In [136]:
# Aggregate sales across all pizza types for each date
aggregated_sales1 = sales_data.groupby(['order_date','pizza_name_id'])['quantity'].sum().reset_index()

In [138]:
aggregated_sales1.set_index('order_date', inplace=True)

In [139]:
aggregated_sales1

,pizza_name_id,quantity
order_date,,
2015-01-01,bbq_ckn_l,6
2015-01-01,bbq_ckn_m,4
2015-01-01,bbq_ckn_s,1
2015-01-01,big_meat_s,5
2015-01-01,calabrese_m,1
...,...,...
2015-12-31,the_greek_l,1
2015-12-31,the_greek_xl,1
2015-12-31,veggie_veg_l,4


In [140]:
# Get unique pizza types
pizza_types = aggregated_sales1['pizza_name_id'].unique()

In [141]:
pizza_types

array(['bbq_ckn_l', 'bbq_ckn_m', 'bbq_ckn_s', 'big_meat_s', 'calabrese_m',
       'cali_ckn_l', 'cali_ckn_m', 'cali_ckn_s', 'ckn_alfredo_m',
       'ckn_alfredo_s', 'ckn_pesto_l', 'ckn_pesto_m', 'classic_dlx_l',
       'classic_dlx_m', 'classic_dlx_s', 'five_cheese_l', 'four_cheese_l',
       'four_cheese_m', 'green_garden_l', 'green_garden_m',
       'green_garden_s', 'hawaiian_l', 'hawaiian_m', 'hawaiian_s',
       'ital_cpcllo_l', 'ital_cpcllo_m', 'ital_supr_l', 'ital_supr_m',
       'ital_supr_s', 'ital_veggie_s', 'mediterraneo_l', 'mediterraneo_m',
       'mexicana_l', 'mexicana_m', 'mexicana_s', 'napolitana_l',
       'napolitana_s', 'pep_msh_pep_l', 'pep_msh_pep_m', 'pep_msh_pep_s',
       'pepperoni_l', 'pepperoni_m', 'pepperoni_s', 'peppr_salami_l',
       'peppr_salami_m', 'peppr_salami_s', 'prsc_argla_l', 'prsc_argla_m',
       'prsc_argla_s', 'sicilian_l', 'sicilian_m', 'sicilian_s',
       'soppressata_l', 'southw_ckn_l', 'southw_ckn_m', 'southw_ckn_s',
       'spicy_ital_

In [142]:
# Store predictions
forecast_results = {}

In [143]:
for pizza_type in pizza_types:
    # Filter data for each pizza type
    pizza_data = aggregated_sales1[aggregated_sales1['pizza_name_id'] == pizza_type].resample('D').sum()

    print(pizza_data)
    
    # Fit SARIMA model
    model = SARIMAX(pizza_data['quantity'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
    model_fit = model.fit(disp=False)
    
    # Forecast the next 7 days
    forecast = model_fit.forecast(steps=7)
    
    # Store the forecast in the results dictionary
    forecast_results[pizza_type] = forecast

           pizza_name_id  quantity
order_date                        
2015-01-01     bbq_ckn_l         6
2015-01-02     bbq_ckn_l         4
2015-01-03     bbq_ckn_l         2
2015-01-04             0         0
2015-01-05     bbq_ckn_l         2
...                  ...       ...
2015-12-27     bbq_ckn_l         1
2015-12-28             0         0
2015-12-29     bbq_ckn_l         1
2015-12-30     bbq_ckn_l         1
2015-12-31     bbq_ckn_l         5

[365 rows x 2 columns]
           pizza_name_id  quantity
order_date                        
2015-01-01     bbq_ckn_m         4
2015-01-02     bbq_ckn_m         7
2015-01-03             0         0
2015-01-04     bbq_ckn_m         2
2015-01-05     bbq_ckn_m         2
...                  ...       ...
2015-12-27     bbq_ckn_m         3
2015-12-28     bbq_ckn_m         1
2015-12-29     bbq_ckn_m         3
2015-12-30     bbq_ckn_m         4
2015-12-31     bbq_ckn_m         4

[365 rows x 2 columns]
           pizza_name_id  quantity
order_d

In [144]:
# Combine results into a DataFrame for easier viewing
forecast_df = pd.DataFrame(forecast_results)
forecast_df

,bbq_ckn_l,bbq_ckn_m,bbq_ckn_s,big_meat_s,calabrese_m,cali_ckn_l,cali_ckn_m,cali_ckn_s,ckn_alfredo_m,ckn_alfredo_s,...,thai_ckn_s,the_greek_l,the_greek_m,spin_pesto_m,veggie_veg_m,mediterraneo_s,soppressata_s,calabrese_s,ital_veggie_l,the_greek_xxl
2015-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.281061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.157433,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.262340,NaN,NaN
2015-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.292006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.360573,NaN,0.075699
2015-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.299372,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.316079,NaN,0.125935
2015-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.359931,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.358349,NaN,0.040552
2015-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.266492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.316674,NaN,0.061932
2015-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.273708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.170057,NaN,0.040830
2015-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.316842,NaN,0.104143
2015-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.128474
2015-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
aggregated_sales2 = aggregated_sales1.copy()

In [151]:
aggregated_sales2.reset_index(inplace=True)
aggregated_sales2

,order_date,pizza_name_id,quantity
0,2015-01-01,bbq_ckn_l,6
1,2015-01-01,bbq_ckn_m,4
2,2015-01-01,bbq_ckn_s,1
3,2015-01-01,big_meat_s,5
4,2015-01-01,calabrese_m,1
...,...,...,...
22959,2015-12-31,the_greek_l,1
22960,2015-12-31,the_greek_xl,1
22961,2015-12-31,veggie_veg_l,4
22962,2015-12-31,veggie_veg_m,2


In [175]:
from prophet import Prophet

# Prepare forecast storage
forecast_results = {}

# Loop over each pizza type
for pizza_type in pizza_types:
    # Filter data for each pizza type
    pizza_data = aggregated_sales2[aggregated_sales2['pizza_name_id'] == pizza_type][['order_date', 'quantity']]
    print(pizza_data)


    # Rename columns to match Prophet's expected format
    pizza_data = pizza_data.rename(columns={'order_date': 'ds', 'quantity': 'y'})
    
    # Fit Prophet model
    model = Prophet()
    model.fit(pizza_data)
    
    # Forecast the next 7 days
    future = model.make_future_dataframe(periods=7)

    forecast = model.predict(future)
    
    # Store forecasted quantities
    forecast_results[pizza_type] = forecast[['ds', 'yhat']].tail(7).set_index('ds')

# Combine forecast results
forecast_df = pd.concat(forecast_results, axis=1)
forecast_df.columns = forecast_df.columns.droplevel(1)  # Remove multi-index for clarity

16:49:57 - cmdstanpy - INFO - Chain [1] start processing
16:49:57 - cmdstanpy - INFO - Chain [1] done processing


      order_date  quantity
0     2015-01-01         6
69    2015-01-02         4
141   2015-01-03         2
255   2015-01-05         2
327   2015-01-06         1
...          ...       ...
22624 2015-12-26         3
22678 2015-12-27         1
22797 2015-12-29         1
22845 2015-12-30         1
22894 2015-12-31         5

[329 rows x 2 columns]
      order_date  quantity
1     2015-01-01         4
70    2015-01-02         7
194   2015-01-04         2
256   2015-01-05         2
328   2015-01-06         8
...          ...       ...
22679 2015-12-27         3
22733 2015-12-28         1
22798 2015-12-29         3
22846 2015-12-30         4
22895 2015-12-31         4

[333 rows x 2 columns]


16:49:57 - cmdstanpy - INFO - Chain [1] start processing
16:49:57 - cmdstanpy - INFO - Chain [1] done processing
16:49:57 - cmdstanpy - INFO - Chain [1] start processing
16:49:57 - cmdstanpy - INFO - Chain [1] done processing


      order_date  quantity
2     2015-01-01         1
71    2015-01-02         1
257   2015-01-05         2
329   2015-01-06         5
533   2015-01-09         3
...          ...       ...
22562 2015-12-24         1
22625 2015-12-26         2
22734 2015-12-28         1
22847 2015-12-30         1
22896 2015-12-31         1

[260 rows x 2 columns]
      order_date  quantity
3     2015-01-01         5
72    2015-01-02         9
142   2015-01-03         3
195   2015-01-04         3
258   2015-01-05         1
...          ...       ...
22680 2015-12-27         5
22735 2015-12-28         3
22799 2015-12-29         6
22848 2015-12-30         3
22897 2015-12-31         6

[357 rows x 2 columns]


16:49:57 - cmdstanpy - INFO - Chain [1] start processing
16:49:57 - cmdstanpy - INFO - Chain [1] done processing
16:49:58 - cmdstanpy - INFO - Chain [1] start processing
16:49:58 - cmdstanpy - INFO - Chain [1] done processing


...

      order_date  quantity
221   2015-01-04         1
289   2015-01-05         2
359   2015-01-06         4
424   2015-01-07         1
495   2015-01-08         1
...          ...       ...
22269 2015-12-19         2
22397 2015-12-21         3
22523 2015-12-23         1
22590 2015-12-24         2
22925 2015-12-31         2

[188 rows x 2 columns]
      order_date  quantity
238   2015-01-04         2
442   2015-01-07         2
578   2015-01-09         1
646   2015-01-10         1
775   2015-01-12         2
...          ...       ...
22416 2015-12-21         2
22541 2015-12-23         1
22715 2015-12-27         1
22778 2015-12-28         2
22945 2015-12-31         1

[206 rows x 2 columns]


16:50:12 - cmdstanpy - INFO - Chain [1] start processing
16:50:12 - cmdstanpy - INFO - Chain [1] done processing
16:50:12 - cmdstanpy - INFO - Chain [1] start processing


      order_date  quantity
262   2015-01-05         1
334   2015-01-06         1
537   2015-01-09         1
601   2015-01-10         1
734   2015-01-12         1
...          ...       ...
21069 2015-11-30         1
21254 2015-12-03         1
21374 2015-12-05         2
21444 2015-12-06         1
22246 2015-12-19         1

[84 rows x 2 columns]


16:50:12 - cmdstanpy - INFO - Chain [1] done processing
16:50:12 - cmdstanpy - INFO - Chain [1] start processing
16:50:12 - cmdstanpy - INFO - Chain [1] done processing


      order_date  quantity
285   2015-01-05         1
356   2015-01-06         1
492   2015-01-08         1
556   2015-01-09         1
693   2015-01-11         1
...          ...       ...
22460 2015-12-22         1
22519 2015-12-23         1
22648 2015-12-26         1
22866 2015-12-30         1
22922 2015-12-31         1

[145 rows x 2 columns]
      order_date  quantity
1677  2015-01-26         1
1746  2015-01-27         1
2822  2015-02-13         1
3011  2015-02-16         1
3590  2015-02-25         1
4712  2015-03-14         1
4917  2015-03-17         1
5177  2015-03-21         1
6663  2015-04-13         1
7312  2015-04-23         1
8352  2015-05-11         1
9120  2015-05-23         1
9611  2015-05-31         1
10754 2015-06-18         1
10944 2015-06-21         1
11843 2015-07-05         1
15107 2015-08-25         1
16881 2015-09-22         1
16944 2015-09-23         1
17204 2015-09-29         1
17536 2015-10-04         1
17932 2015-10-10         1
18789 2015-10-24         1
1969

In [176]:
forecast_df

,bbq_ckn_l,bbq_ckn_m,bbq_ckn_s,big_meat_s,calabrese_m,cali_ckn_l,cali_ckn_m,cali_ckn_s,ckn_alfredo_m,ckn_alfredo_s,...,thai_ckn_s,the_greek_l,the_greek_m,spin_pesto_m,veggie_veg_m,mediterraneo_s,soppressata_s,calabrese_s,ital_veggie_l,the_greek_xxl
ds,,,,,,,,,,,,,,,,,,,,,
2015-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.038039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.089411,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071836,NaN,NaN
2015-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.231009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.317575,NaN,1.0
2015-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.326629,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.244534,NaN,1.0
2015-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.081427,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.301316,NaN,1.0
2015-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.023417,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.338998,NaN,1.0
2015-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.955928,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.250218,NaN,1.0
2015-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.290797,NaN,1.0
2015-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [178]:
forecast_df.reset_index(inplace=True)

In [185]:
drop_col = forecast_df.iloc[0:12]

In [186]:
drop_col

,ds,bbq_ckn_l,bbq_ckn_m,bbq_ckn_s,big_meat_s,calabrese_m,cali_ckn_l,cali_ckn_m,cali_ckn_s,ckn_alfredo_m,...,thai_ckn_s,the_greek_l,the_greek_m,spin_pesto_m,veggie_veg_m,mediterraneo_s,soppressata_s,calabrese_s,ital_veggie_l,the_greek_xxl
0,2015-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071836,NaN,NaN
2,2015-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.317575,NaN,1.0
3,2015-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.244534,NaN,1.0
4,2015-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.301316,NaN,1.0
5,2015-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.338998,NaN,1.0
6,2015-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.250218,NaN,1.0
7,2015-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.290797,NaN,1.0
8,2015-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,2015-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
forecast_df.drop(forecast_df.index[0:13], axis=0, inplace=True)

In [191]:
forecast_df

,ds,bbq_ckn_l,bbq_ckn_m,bbq_ckn_s,big_meat_s,calabrese_m,cali_ckn_l,cali_ckn_m,cali_ckn_s,ckn_alfredo_m,...,thai_ckn_s,the_greek_l,the_greek_m,spin_pesto_m,veggie_veg_m,mediterraneo_s,soppressata_s,calabrese_s,ital_veggie_l,the_greek_xxl
13,2016-01-01,3.039892,3.523353,1.683330,5.712413,2.065241,3.418103,3.231436,2.254391,2.552646,...,1.892600,1.779655,1.370404,1.469654,2.282200,1.646629,1.534859,NaN,1.434963,NaN
14,2016-01-02,3.144230,2.726515,1.578059,5.764076,2.036299,2.758662,2.651827,1.690694,2.359060,...,1.926125,1.419702,1.414494,1.920842,2.215857,1.573285,1.393112,NaN,1.361787,NaN
15,2016-01-03,2.440990,2.739042,1.532595,5.018439,1.786695,2.664383,2.321365,1.816321,2.102049,...,1.723501,1.235302,1.529766,1.557803,1.900514,1.585760,1.499545,NaN,1.336318,NaN
16,2016-01-04,2.312898,2.692902,1.682236,5.218070,2.154261,2.667289,2.536999,1.774430,2.396977,...,2.159306,1.492343,1.476678,1.308941,1.901095,1.945472,1.467783,NaN,1.077965,NaN
17,2016-01-05,2.401410,2.783561,2.074992,5.227992,2.026799,2.801499,2.246670,1.697276,2.210475,...,1.977643,1.373973,1.497215,1.592111,1.709106,1.562072,1.378053,NaN,1.337172,NaN
18,2016-01-06,2.349312,2.890167,1.595415,5.270116,2.288133,2.972542,2.360583,1.745667,2.114530,...,1.672207,1.410401,1.395254,1.366078,2.081166,1.801363,1.285417,NaN,1.325906,NaN
19,2016-01-07,3.085276,3.097434,1.821236,5.564690,2.013199,2.810778,2.887367,1.731209,2.647292,...,NaN,1.328262,NaN,1.359490,2.263824,1.372730,1.382745,NaN,1.559471,NaN
